In [1]:
import re
import asyncio
import yaml

from typing import List, Set, Dict


import aiohttp
from aiohttp import ClientSession

from time import sleep
import selenium
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Firefox
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from adbot.misc.website import Website, URL
from adbot import HEADERS

In [2]:
site = URL("www.tv2.dk")
site2 = URL("tv2.dk")
site3 = URL("www.tv2.dk")

In [3]:
l = [site, site2, site3]

In [4]:
set(l)

{URL<https://tv2.dk>, URL<https://www.tv2.dk>}

In [3]:
site + "/new"

In [6]:
site

URL<https://www.tv2.dk/new>

In [5]:
resp = requests.get(site)

In [6]:
website = Website(site)

In [7]:
website.netloc

'tv2.dk'

In [10]:
website.domain

'www.tv2.dk'

In [11]:
site.url

'https://www.tv2.dk/new'

## YAML

In [12]:
with open("adbot/settings/default.yml", mode="r", encoding="utf-8") as file:
    print(yaml.safe_load(file))

{'version': 0.1, 'environment': 'development', 'files': [{'image': ['.png', '.jpg', '.jpeg', '.svg']}, {'video': ['.mp3', '.mp4']}], 'tags': ['iframe']}


In [10]:
file = yaml.SafeLoader("adbot/settings/default.yml")

In [6]:
print(file)

## Requests

In [22]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
url = "https://tv2.dk/"

In [13]:
response = requests.get(url)

In [14]:
html = BeautifulSoup(response.text)

In [15]:
def validate_link(link: str, domain: str) -> str:
    length = len("https://")
    if bool(re.search(domain, link)):
        if link[:length] == "https":
            return link
        else:
            return "https://" + domain + link
    return None

In [16]:
link = 'https://tv2.dk/navigation'
#link = "main"

In [17]:
match = bool(re.search("tv2.dk", link))

In [18]:
match

True

In [19]:
list(i.get("href") for i in html.find_all("a"))

['#main',
 'https://tv2.dk',
 'https://tv2.dk/navigation',
 'https://mit.tv2.dk',
 'https://search.tv2.dk/',
 'https://play.tv2.dk',
 'https://nyheder.tv2.dk/',
 'https://vejr.tv2.dk/',
 'https://sport.tv2.dk/',
 'https://tv.tv2.dk/',
 'https://nyheder.tv2.dk/live/2022-05-26-seneste-nyt',
 'https://nyheder.tv2.dk/live/2022-05-26-seneste-nyt#entry=3799930',
 'https://sport.tv2.dk/tennis/2022-05-26-holger-rune-nyt-efter-slem-forskraekkelse',
 'https://nyheder.tv2.dk/lokalt/2022-05-26-stort-festival-vartegn-oedelagt-kommer-ikke-op-igen',
 'https://nyheder.tv2.dk/udland/2022-05-26-russiske-lastbiler-laesset-med-propaganda-er-rullet-ind-i-mariupol',
 'https://nyheder.tv2.dk/live/2022-05-26-seneste-nyt-om-krigen-i-ukraine-26-maj',
 'https://nyheder.tv2.dk/udland/2022-05-26-rusland-kommer-med-trussel-efter-blokering-fra-youtube',
 'https://nyheder.tv2.dk/udland/2022-05-25-ungarn-erklaerer-undtagelsestilstand-som-foelge-af-krigen',
 'https://sport.tv2.dk/live/2022-05-26-pokalfinale-fc-midtjyll

In [20]:
urls = set(i.get("href") for i in html.find_all("a") if validate_link(i.get("href"), "tv2.dk") is not None)

In [36]:
len(urls)

100

In [37]:
%%time

async def fetch_link(url: str, session: ClientSession) -> str:
    print("Fetch", url)
    response = await session.request(method="GET", url=url)
    print("Got response from", url)
    html = await response.text()
    return html


async def collect_links(html: str) -> List[str]:
    href = re.compile(r'href="(.*?)"')
    _links = list(link for link in href.findall(html))
    return _links


async def fetch_links(urls: List[str]) -> List[str]:
    """
    Usage:
        output = asyncio.run(fetch_links(urls=urls))
    """
    links = list()

    async def update_links(url: str, session: ClientSession) -> None:
        global links
        html = await fetch_link(url=url, session=session)
        _links = await collect_links(html=html)
        print("Update", url)
        links += links + _links

    async with ClientSession() as session:
        tasks = list(update_links(url=url, session=session) for url in urls)
        await asyncio.gather(*tasks)

    return links

output = asyncio.run(fetch_html(urls=urls))

Fetch https://privatliv.tv2.dk/
Fetch https://sport.tv2.dk/live/2022-05-26-pokalfinale-fc-midtjylland-ob
Fetch https://sport.tv2.dk/fodbold/2022-05-25-daarligt-zlatan-nyt-faar-hjemmeside-til-at-gaa-ned
Fetch https://tv.tv2.dk/soger
Fetch https://tv.tv2.dk/
Fetch https://nyheder.tv2.dk/samfund/2022-05-25-sundhedsministeriet-brostrom-far-ikke-topjob-alligevel
Fetch https://nyheder.tv2.dk/samfund/2022-05-25-tivoli-forklarer-aflysning-af-koncerter-vi-vil-gerne-vaere-sikre-paa-at-alle-faar
Fetch https://nyheder.tv2.dk/politik/2022-05-26-minister-misbrugte-kreditkort-pa-vaertshus-og-smorrebrod
Fetch https://nyheder.tv2.dk/krimi/2022-05-26-lig-fundet-gravet-ned-ved-kolding
Fetch https://nyheder.tv2.dk/lokalt/2022-05-25-efterskoler-oplever-fordobling-i-frafald
Fetch https://vejr.tv2.dk/2022-05-25-koeligt-foraarsvejr-paa-vej
Fetch https://nyheder.tv2.dk/samfund/2022-05-25-elever-paa-herlufsholm-udsat-for-trusler-elev-vaeltet-paa-cyklen
Fetch https://nyheder.tv2.dk/politik/2022-05-25-heunicke-fo

Got response from https://play.tv2.dk
Got response from https://sport.tv2.dk/live/2022-05-26-pokalfinale-fc-midtjylland-ob
Got response from https://tv.tv2.dk/
Got response from https://sport.tv2.dk/tennis/2022-05-26-holger-rune-nyt-efter-slem-forskraekkelse
Got response from https://sport.tv2.dk/fodbold/2022-05-25-bekymret-fan-stillede-spoergsmaal-hjulmand-svarede-med-kegler
Got response from https://sport.tv2.dk/fodbold/2022-05-26-spillere-overraskede-mourinho-paa-pressemoede-efter-triumf
Got response from https://tv.tv2.dk/2017-08-12-tv-2-apps-tv-2-play-nyheder-sport-vejr-og-tv
Got response from https://sport.tv2.dk/fodbold/2022-05-25-de-naeste-par-dage-er-vi-alle-skuffede-siger-frank-arnesen
Got response from https://sport.tv2.dk/tennis/2022-05-26-rune-buldrer-videre-ved-french-open-trods-forskraekkelse
Got response from https://sport.tv2.dk/fodbold/2022-05-26-zlatan-har-taget-smertestillende-hver-dag-i-et-halvt-aar
Got response from https://nyheder.tv2.dk/krimi/2022-05-26-lig-fund

Update https://sport.tv2.dk/fodbold/2022-05-25-bekymret-fan-stillede-spoergsmaal-hjulmand-svarede-med-kegler
Update https://sport.tv2.dk/fodbold/2022-05-26-england-skal-spille-for-tomme-tribuner-en-ydmygelse-siger-landstraener
Update https://omtv2.tv2.dk/kontakt/kontakt-tv-2/
Update https://nyheder.tv2.dk/2022-05-26-regeringen-ma-traekke-lovforslag-om-sociale-medier-tilbage
Got response from https://nyheder.tv2.dk/live/2022-05-26-seneste-nyt
Update https://tv2.dk/navigation
Update https://kundeservice.tv2.dk/
Update https://nyheder.tv2.dk/politik/2022-05-11-alt-du-skal-vide-om-forsvarsforbeholdet
Update https://nyheder.tv2.dk/politik/2022-05-25-heunicke-forventer-oeget-smitte-med-abekopper-men-nedtoner-risiko
Update https://tv.tv2.dk/2017-08-12-tv-2-apps-tv-2-play-nyheder-sport-vejr-og-tv
Update https://nyheder.tv2.dk/samfund/2022-05-26-bramsen-vil-bremse-vrede-bilister-allerede-pa-koreskolen
Update https://nyheder.tv2.dk/politik/2022-05-24-store-dele-af-europa-er-ret-ligeglad-med-dans

Got response from https://nyheder.tv2.dk/1234
Update https://nyheder.tv2.dk/samfund/2022-05-26-overlaege-finder-vaccinetilbud-forsvarligt-selvom-godkendelse-mangler
Update https://nyheder.tv2.dk/live/2022-05-26-seneste-nyt-om-krigen-i-ukraine-26-maj
Got response from https://nyheder.tv2.dk/samfund/2022-05-26-hjemmeside-udnyttede-forbrugerne-ved-at-lokke-med-billig-el
Update https://nyheder.tv2.dk/1234
Update https://nyheder.tv2.dk/samfund/2022-05-26-hjemmeside-udnyttede-forbrugerne-ved-at-lokke-med-billig-el
Got response from https://omtv2.tv2.dk/job/ledige-stillinger
Update https://omtv2.tv2.dk/job/ledige-stillinger
Got response from https://omtv2.tv2.dk/presse/
Update https://omtv2.tv2.dk/presse/
CPU times: user 628 ms, sys: 87.9 ms, total: 716 ms
Wall time: 2.59 s


In [34]:
output["https://tv2.dk"]

['https://free-cdn.tv2i.dk',
 'https://scripts.tv2a.dk',
 'https://web.tv2a.dk',
 'https://decks.services.tv2.dk',
 'https://fonts.tv2a.dk/v8/AlrightSansLT-Regular.woff2',
 'https://fonts.tv2a.dk/v8/AlrightSansLT-RegularItalic.woff2',
 'https://fonts.tv2a.dk/v8/AlrightSansLT-Bold.woff2',
 'https://fonts.tv2a.dk/v8/AlrightSansLT-BoldItalic.woff2',
 'https://fonts.tv2a.dk/v8/AlrightSansLT-Black.woff2',
 'https://fonts.tv2a.dk/v8/AlrightSansLT-BlackItalic.woff2',
 'https://web.tv2a.dk/static/images/favicon.png',
 'https://web.tv2a.dk/static/images/apple-touch-icon.png',
 'https://web.tv2a.dk/css/Link.bde491ad6e22de0c88a44f58c206b40a.css',
 'https://web.tv2a.dk/css/TV2dkHeader.75f87010c18ce85b740e63e38cae6edb.css',
 'https://web.tv2a.dk/css/Deck.25ea5befa817662c5c4bd54e7ac8a5e8.css',
 'https://scripts.tv2a.dk/cmp.cFjb.css',
 'https://web.tv2a.dk/css/Ad.196a4de83633afda822b987a779a3a71.css',
 'https://web.tv2a.dk/css/Grid.453a6ada970be062e25baa9e4c62c13e.css',
 'https://web.tv2a.dk/css/Head

In [26]:
type(html)

str

In [128]:
html.find_all("a")

[<a class="tc_header__skip__link" href="#main">Hop til indhold</a>,
 <a aria-label="Gå til forsiden af tv2.dk" class="tc_header__logo" href="https://tv2.dk"><svg class="tc_icon__tv2logo" height="40" viewbox="0 0 90 40" width="90" xmlns="http://www.w3.org/2000/svg"><title>TV 2 Logo</title><path d="M89.992 10.172c.3 7.647-8.03 14.59-16.056 14.59h-20.62l5.656-8.324c.44-.663 1.464-1.2 2.274-1.2H75.57c1.847 0 3.85-1.693 3.666-3.555-.155-1.495-1.364-2.144-2.657-2.16H44.55L50.205 1.2C50.66.535 51.668 0 52.478 0h26.26c5.657 0 11.014 4.135 11.254 10.172zm-51.81-1.847L43.84 0H7.93c-.797 0-1.82.536-2.274 1.2L0 9.522h35.91c.81 0 1.82-.535 2.273-1.198zM17.877 40h16.27c.796 0 1.82-.535 2.274-1.2l16-23.562h-10.3c-.797 0-1.82.537-2.275 1.2L25.763 37.164l2.33-21.926h-10.67L15.12 36.966C14.95 38.646 16.17 40 17.876 40zm61.076-1.213l5.655-8.325H50.9c-.795 0-1.818.538-2.272 1.2l-5.657 8.325H76.68c.81.013 1.833-.522 2.274-1.2z"></path></svg></a>,
 <a aria-controls="navigation-overlay" aria-expanded="false"

In [27]:
HREF_RE = re.compile(r'href="(.*?)"')

In [28]:
for link in HREF_RE.findall(html):
    print(link)

https://free-cdn.tv2i.dk
https://scripts.tv2a.dk
https://web.tv2a.dk
https://decks.services.tv2.dk
https://fonts.tv2a.dk/v8/AlrightSansLT-Regular.woff2
https://fonts.tv2a.dk/v8/AlrightSansLT-RegularItalic.woff2
https://fonts.tv2a.dk/v8/AlrightSansLT-Bold.woff2
https://fonts.tv2a.dk/v8/AlrightSansLT-BoldItalic.woff2
https://fonts.tv2a.dk/v8/AlrightSansLT-Black.woff2
https://fonts.tv2a.dk/v8/AlrightSansLT-BlackItalic.woff2
https://web.tv2a.dk/static/images/favicon.png
https://web.tv2a.dk/static/images/apple-touch-icon.png
https://web.tv2a.dk/css/Link.bde491ad6e22de0c88a44f58c206b40a.css
https://web.tv2a.dk/css/TV2dkHeader.75f87010c18ce85b740e63e38cae6edb.css
https://web.tv2a.dk/css/Deck.25ea5befa817662c5c4bd54e7ac8a5e8.css
https://scripts.tv2a.dk/cmp.cFjb.css
https://web.tv2a.dk/css/Ad.196a4de83633afda822b987a779a3a71.css
https://web.tv2a.dk/css/Grid.453a6ada970be062e25baa9e4c62c13e.css
https://web.tv2a.dk/css/Heading.3683c3b9dc5763e3c86106714e659bca.css
https://web.tv2a.dk/css/Teaser.ca

In [66]:
html = output[0]

In [61]:
html

<ClientResponse(https://play.tv2.dk) [200 OK]>
<CIMultiDictProxy('Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Thu, 26 May 2022 12:41:40 GMT', 'Etag': '"34e93-DjIEcK34mYivvz6NAStuoGRQBeE"', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=Edge', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-Cache-Status': 'HIT', 'Cache-Control': 'private, max-age=0', 'Vary': 'Accept-Encoding', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 060df07995f24318e95556d506f04e12.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'CPH50-C1', 'X-Amz-Cf-Id': 'hYAke4WsIn-l7tTgBITMk3xZLKA1ErtFhiBMIZIIuP_PyHe056Uwbw==')>

## Selenium

https://stackoverflow.com/questions/15018372/how-to-take-partial-screenshot-with-selenium-webdriver-in-python

In [2]:
url = "https://nyheder.tv2.dk/udland/2022-05-22-rusland-saetter-terminatoren-ind-i-krigen-i-ukraine"

In [13]:
driver = Firefox()

In [14]:
driver.get(url)

In [44]:
#driver.find_element(by=By.CLASS_NAME, value='adform-adbox-static')

In [30]:
driver.find_element(by=By.XPATH, value="/html/body/div[1]/div/div/div/div[2]/div/button[3]").click()

## Find by iframes

In [92]:
fox = webdriver.Firefox()
fox.get('http://tv2.dk/')
sleep(3)
fox.find_element(by=By.XPATH, value="/html/body/div[1]/div/div/div/div[2]/div/button[3]").click()

In [113]:
fox.find_elements_by_tag_name('iframe')

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_97515/685088341.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  fox.find_elements_by_tag_name('iframe')


[<selenium.webdriver.remote.webelement.WebElement (session="f4a4b0e0-3df3-42c2-8dab-3ae24324e701", element="51b6d0ef-736b-447b-915b-4c1643b11b7c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4a4b0e0-3df3-42c2-8dab-3ae24324e701", element="613cc16f-7c2b-4138-b799-ed73f36ab9c7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4a4b0e0-3df3-42c2-8dab-3ae24324e701", element="acdda797-9aa1-4d86-a256-d65172dedeec")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4a4b0e0-3df3-42c2-8dab-3ae24324e701", element="3638d6c0-8c9c-4a9e-a90d-81e9b5c7140d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4a4b0e0-3df3-42c2-8dab-3ae24324e701", element="d6607d69-00af-4317-a5e4-ae2618695b89")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4a4b0e0-3df3-42c2-8dab-3ae24324e701", element="a7498eb9-3957-454c-a5e7-efa79d567bf9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4a4b0e0-3df3-42c2-8dab-3ae24324e701", element

In [108]:
fox.switch_to.frame(fox.find_element_by_tag_name("iframe"))

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_97515/2859000034.py:1: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  fox.switch_to.frame(fox.find_element_by_tag_name("iframe"))


In [109]:
element = fox.find_element_by_class_name("dcmads")
element.screenshot("s.png")

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_97515/2822687994.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  element = fox.find_element_by_class_name("dcmads")


True

In [110]:
fox.switch_to.default_content()

### Screenshot

In [72]:
from selenium import webdriver
from PIL import Image
from io import StringIO, BytesIO
import base64

fox = webdriver.Firefox()
fox.get('http://tv2.dk/')
sleep(3)
fox.find_element(by=By.XPATH, value="/html/body/div[1]/div/div/div/div[2]/div/button[3]").click()

element = fox.find_element(by=By.CLASS_NAME, value='tc_icon__play') # find part of the page you want image of
location = element.location
size = element.size
element.screenshot('screenshot.png')

True

In [83]:
element = fox.find_element(by=By.CLASS_NAME, value='adsm-skin-banner-parent')

NoSuchElementException: Message: Unable to locate element: .adnm-banner
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16


In [82]:
element.screenshot("screenshot.png")

True